In [1]:
# !pip install -q stop-words

import csv
import tensorflow as tf

from tensorflow.keras.utils import pad_sequences

from Helper import *

In [2]:
path = "/tmp/bbc-text.csv"

t = []; l = []

with open(path, "r") as fileread:
  reader = csv.reader(fileread, delimiter=",")
  next(reader)
  for r in reader:
    l.append(r[0])
    t.append(stopper(r[1]))

len(t), len(l)

(2225, 2225)

In [3]:
train_split = 0.8

lt = int(train_split * len(t))

train = t[:lt]; testi = t[lt:]

len(train), len(testi)

(1780, 445)

In [4]:
train_label = l[:lt]; testi_label = l[lt:]

len(train_label), len(testi_label)

(1780, 445)

In [5]:
token = fit_tokener(train, 1000, "<OOV>")

ti = token.word_index

len(ti)

27284

In [6]:
def padded(sentences, token, padding, maxlen):
  i = token.texts_to_sequences(sentences)
  i = pad_sequences(i, maxlen=maxlen, padding=padding)
  return i

train = padded(train, token, "post", 120)

train.shape

(1780, 120)

In [7]:
testi = padded(testi, token, "post", 120)

testi.shape

(445, 120)

In [8]:
import numpy as np

def label_token(all, split):
  lt = Tokenizer()
  lt.fit_on_texts(all)
  label_seq = lt.texts_to_sequences(split)
  result = np.array(label_seq) - 1
  return result

train_l = label_token(l, train_label)

list(train_l[:5])

[array([3]), array([1]), array([0]), array([0]), array([4])]

In [9]:
testi_l = label_token(l, testi_label)

list(testi_l[:5])

[array([4]), array([3]), array([2]), array([0]), array([0])]

In [10]:
from tensorflow.keras import layers

x = layers.Embedding(1000, 16, input_length=120)

i = x(train)

i.shape

TensorShape([1780, 120, 16])

In [11]:
i[0][0].numpy()

array([-0.00325422,  0.00261511, -0.01418906, -0.04071516, -0.00706217,
       -0.01953771,  0.04606614, -0.03638284,  0.04723959,  0.01349563,
        0.00644062, -0.02367474, -0.01062205,  0.02102679,  0.01827529,
       -0.01350601], dtype=float32)

In [12]:
i[0][0].numpy().mean()

-0.0008615501

In [13]:
i[1][0].numpy().mean()

0.0115881115

In [14]:
x = layers.GlobalAveragePooling1D()

i = x(i)

i.shape

TensorShape([1780, 16])

In [15]:
i[0].numpy()

array([-0.01379436,  0.01266018, -0.00956842, -0.00896214,  0.00519044,
        0.00338324,  0.01240792,  0.01691397, -0.00876355, -0.00129155,
       -0.00841337,  0.00691035,  0.02139853, -0.01746516, -0.00574047,
        0.01504128], dtype=float32)

In [16]:
i[1].numpy()

array([-0.02364091,  0.01094848, -0.01201959, -0.00567404,  0.00686344,
        0.00447548,  0.01557768,  0.02097865, -0.01039583,  0.00046095,
       -0.00791535,  0.00907364,  0.0260924 , -0.02299085,  0.00158765,
        0.01745329], dtype=float32)

In [17]:
x = layers.Dense(24, activation="relu")

i = x(i)

i.shape

TensorShape([1780, 24])

In [18]:
i[0].numpy()

array([0.        , 0.01311627, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.00157289, 0.        , 0.01665939,
       0.01526596, 0.00094323, 0.01612446, 0.        , 0.        ,
       0.01945109, 0.00470434, 0.        , 0.02337977, 0.01040706,
       0.00013561, 0.00246948, 0.        , 0.02204542], dtype=float32)

In [19]:
x = layers.Dense(5, activation="softmax")

i = x(i)

i.shape

TensorShape([1780, 5])

In [20]:
i[0].numpy()

array([0.20167449, 0.20311934, 0.19818482, 0.19624476, 0.20077653],
      dtype=float32)

In [ ]:
model = tf.keras.models.Sequential([
    layers.Embedding(1000, 16, input_length=120),
    layers.GlobalAveragePooling1D(),
    layers.Dense(24, activation="relu"),
    layers.Dense(5, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

hist = model.fit(train, train_l, validation_data=(testi, testi_l), epochs=10)